In [1]:
import json
from transformers import pipeline
import pandas as pd
from tqdm import tqdm


/root/anaconda3/envs/SBERT-WK/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

# classifier = pipeline("text-classification", model = "roberta-large-mnli")
classifier = pipeline("text-classification", model = "microsoft/deberta-xlarge-mnli")

In [3]:
df = pd.read_json('/root/thesis/ViLT/cosmos/test_data.json',orient="records", lines=True)

In [4]:
df = pd.read_json('./nli-data.json',orient="records")

In [38]:
classifier(
    """
    Vladimir Putin said that 'ISIS will regret' assassinating a Russian ambassador since he is 'not Obama' and this is 'not Benghazi.' is fake.A quote ostensibly uttered by Russian president Vladimir Putin criticizing President Obama is a fabricated one.
    """
)[0]['label']

'ENTAILMENT'

In [4]:
tqdm.pandas()
df['nli_label'] = df.progress_apply(lambda x: classifier(x.caption1+x.caption2)[0], axis=1)
df['nli_label_reverse'] = df.progress_apply(lambda x: classifier(x.caption2+x.caption1)[0], axis=1)

100%|██████████| 1700/1700 [09:48<00:00,  2.89it/s]


In [5]:
df.to_json('nli-data.json',orient='records')

In [14]:
converter = lambda x : 1 if x.nli_label['label'] == 'CONTRADICTION' else (0 if x.nli_label['label'] == 'ENTAILMENT' else (1 if x.bert_base_score < 0.5 else 0))
df['predict'] =df.progress_apply(converter, axis=1) 
confusion_matrix = pd.crosstab(df['predict'], df['context_label'], rownames=['Predicted'], colnames=['Actual'])
print(confusion_matrix)
result = (confusion_matrix[0][0]+confusion_matrix[1][1])/1700
print('accuracy:', result)

100%|██████████| 1700/1700 [00:00<00:00, 45868.29it/s]

Actual       0    1
Predicted          
0          597   70
1          253  780
accuracy: 0.81


In [7]:
converter = lambda x : 1 if (x.nli_label['label'] == 'CONTRADICTION') else 0
df['predict'] =df.progress_apply(converter, axis=1) 
confusion_matrix = pd.crosstab(df['predict'], df['context_label'], rownames=['Predicted'], colnames=['Actual'])
print(confusion_matrix)
result = confusion_matrix[1][1]/(confusion_matrix[0][1]+confusion_matrix[1][1])
print('accuracy:', result)

100%|██████████| 1700/1700 [00:00<00:00, 73215.56it/s]

Actual       0    1
Predicted          
0          808  470
1           42  380
accuracy: 0.9004739336492891


In [6]:
converter = lambda x : 1 if (x.nli_label_reverse['label'] == 'CONTRADICTION') else 0
df['predict'] =df.progress_apply(converter, axis=1) 
confusion_matrix = pd.crosstab(df['predict'], df['context_label'], rownames=['Predicted'], colnames=['Actual'])
print(confusion_matrix)
result = confusion_matrix[1][1]/(confusion_matrix[0][1]+confusion_matrix[1][1])
print('accuracy:', result)

100%|██████████| 1700/1700 [00:00<00:00, 71989.23it/s]

Actual       0    1
Predicted          
0          817  511
1           33  339
accuracy: 0.9112903225806451


In [19]:
converter = lambda x : 1 if (x.nli_label_reverse['label'] == 'CONTRADICTION' and x.nli_label['label'] == 'CONTRADICTION') else 0
df['predict'] =df.progress_apply(converter, axis=1) 
confusion_matrix = pd.crosstab(df['predict'], df['context_label'], rownames=['Predicted'], colnames=['Actual'])
print(confusion_matrix)
result = confusion_matrix[1][1]/(confusion_matrix[0][1]+confusion_matrix[1][1])
print('accuracy:', result)

100%|██████████| 1700/1700 [00:00<00:00, 67505.96it/s]

Actual       0    1
Predicted          
0          833  588
1           17  262
accuracy: 0.9390681003584229


In [8]:
converter = lambda x : 0 if (x.nli_label['label'] == 'ENTAILMENT') else 1
df['predict'] =df.progress_apply(converter, axis=1) 
confusion_matrix = pd.crosstab(df['predict'], df['context_label'], rownames=['Predicted'], colnames=['Actual'])
print(confusion_matrix)
result = confusion_matrix[1][1]/(confusion_matrix[0][1]+confusion_matrix[1][1])
print('accuracy:', result)

100%|██████████| 1700/1700 [00:00<00:00, 74706.03it/s]

Actual       0    1
Predicted          
0          220    4
1          630  846
accuracy: 0.573170731707317


In [9]:
converter = lambda x : 0 if (x.nli_label_reverse['label'] == 'ENTAILMENT') else 1
df['predict'] =df.progress_apply(converter, axis=1) 
confusion_matrix = pd.crosstab(df['predict'], df['context_label'], rownames=['Predicted'], colnames=['Actual'])
print(confusion_matrix)
result = confusion_matrix[1][1]/(confusion_matrix[0][1]+confusion_matrix[1][1])
print('accuracy:', result)

100%|██████████| 1700/1700 [00:00<00:00, 73193.01it/s]

Actual       0    1
Predicted          
0          146   23
1          704  827
accuracy: 0.5401698236446767


In [18]:
converter = lambda x : 0 if (x.nli_label_reverse['label'] == 'ENTAILMENT' or x.nli_label['label'] == 'ENTAILMENT') else 1
df['predict'] =df.progress_apply(converter, axis=1) 
confusion_matrix = pd.crosstab(df['predict'], df['context_label'], rownames=['Predicted'], colnames=['Actual'])
print(confusion_matrix)
result = confusion_matrix[1][1]/(confusion_matrix[0][1]+confusion_matrix[1][1])
print('accuracy:', result)

100%|██████████| 1700/1700 [00:00<00:00, 53038.35it/s]

Actual       0    1
Predicted          
0          324   27
1          526  823
accuracy: 0.6100815418828762


In [15]:
predict = df[df['predict'] == 0]
predict = predict[predict['context_label'] == 1]

In [16]:
predict

,img_local_path,caption1,caption2,context_label,article_url,maskrcnn_bboxes,caption1_modified,caption1_entities,caption2_modified,caption2_entities,bert_base_score,bert_large_score,nli_label,nli_label_reverse,predict
49,test/49.jpg,The FBI raided the CDC in the middle of the ni...,A known hoax purveyor spread false claims that...,1,https://www.snopes.com/fact-check/fbi-raids-cd...,"[[641.7175903320312, 325.96148681640625, 672.7...",The ORG raided the ORG in TIME to seize data o...,"[[FBI, ORG], [CDC, ORG], [the middle of the ni...",A known hoax purveyor spread false claims that...,"[[FBI, ORG], [CDC, ORG], [the middle of the ni...",0.692395,0.797202,"{'label': 'NEUTRAL', 'score': 0.7838650345802307}","{'label': 'ENTAILMENT', 'score': 0.67717039585...",0
53,test/53.png,A photograph shows a genuine fossil of a megal...,While this image likely shows a genuine megalo...,1,https://www.snopes.com/fact-check/megalodon-to...,"[[137.79473876953125, 10.073151588439941, 455....",A photograph shows a genuine fossil of a megal...,[],While this image likely shows a genuine megalo...,"[[two, CARDINAL]]",0.712833,0.711089,"{'label': 'NEUTRAL', 'score': 0.8463303446769714}","{'label': 'ENTAILMENT', 'score': 0.76704019308...",0
55,test/55.jpg,Trash being left behind by the migrant caravan.,This photograph was not recent and did not sho...,1,https://www.snopes.com/fact-check/refugee-cara...,"[[492.66387939453125, 337.4176940917969, 819.8...",Trash being left behind by the migrant caravan.,[],This photograph was not recent and did not sho...,"[[Honduras, GPE], [U.S, GPE], [Mexico, GPE], [...",0.339660,0.161456,"{'label': 'CONTRADICTION', 'score': 0.89957839...","{'label': 'ENTAILMENT', 'score': 0.90851658582...",0
76,test/76.jpg,A picture showing Goldie Hawn and Kurt Russel...,An image of actors Goldie Hawn and Kurt Russel...,1,https://www.snopes.com/fact-check/goldie-hawn-...,"[[157.21160888671875, 8.691365242004395, 276.2...",A picture showing PERSON and PERSON both wear...,"[[Goldie Hawn, PERSON], [Kurt Russell, PERSON]...",An image of actors PERSON and PERSON wearing P...,"[[Goldie Hawn, PERSON], [Kurt Russell, PERSON]...",0.695945,0.723471,"{'label': 'NEUTRAL', 'score': 0.7818806171417236}","{'label': 'ENTAILMENT', 'score': 0.95883262157...",0
200,test/200.jpg,Actors Goldie Hawn and Kurt Russell were spott...,A picture purportedly showing Goldie Hawn and ...,1,https://www.snopes.com/fact-check/goldie-hawn-...,"[[73.01750183105469, 1.932313203811645, 382.68...",Actors Goldie Hawn and PERSON were spotted wea...,"[[Kurt Russell, PERSON], [Donald Trump, PERSON]]",A picture purportedly showing PERSON and PERSO...,"[[Goldie Hawn, PERSON], [Kurt Russell, PERSON]...",0.777599,0.736808,"{'label': 'NEUTRAL', 'score': 0.8834425806999207}","{'label': 'ENTAILMENT', 'score': 0.88628125190...",0
204,test/204.jpg,California's governor is under investigation f...,The story California Governor Jerry Brown to a...,1,https://www.snopes.com/fact-check/jerry-brown-...,"[[304.325927734375, 311.4779052734375, 357.188...",GPE's governor is under investigation for poss...,"[[California, GPE]]",The story GPE Governor PERSON to a corruption ...,"[[California, GPE], [Jerry Brown, PERSON]]",0.483953,0.461200,"{'label': 'CONTRADICTION', 'score': 0.50741523...","{'label': 'ENTAILMENT', 'score': 0.87024801969...",0
224,test/224.png,A picture shows a massive tsunami headed for a...,The pictures of the huge tsunami hitting a cro...,1,https://www.snopes.com/fact-check/huge-tsunami...,"[[427.2711181640625, 322.89752197265625, 441.5...",A picture shows a massive tsunami headed for a...,[],The pictures of the huge tsunami hitting a cro...,"[[Honolulu, GPE]]",0.496911,0.476031,"{'label': 'NEUTRAL', 'score': 0.9402892589569092}","{'label': 'ENTAILMENT', 'score': 0.96437042951...",0
296,test/296.jpg,President Trump signed an executive order nami...,The post about Trump signeing an executive ord...,1,https://www.snopes.com/fact-check/trump-order-...,"[[364.7497863769531, 316.35552978

In [51]:
predict = df[df['predict'] == 1]
predict = predict[predict['context_label'] == 0]

In [11]:
converter = lambda x : 0 if x.nli_label['label'] == 'ENTAILMENT' else 1
df['predict'] =df.progress_apply(converter, axis=1) 
confusion_matrix = pd.crosstab(df['predict'], df['context_label'], rownames=['Predicted'], colnames=['Actual'])
print(confusion_matrix)
result = (confusion_matrix[0][0]+confusion_matrix[1][1])/1700
print('accuracy:', result)

100%|██████████| 1700/1700 [00:00<00:00, 59007.24it/s]

Actual       0    1
Predicted          
0          220    4
1          630  846
accuracy: 0.6270588235294118


In [9]:
converter = lambda x : 0 if x.nli_label['label'] == 'ENTAILMENT' else (1 if x.bert_base_score < 0.5 else 0)
df['predict'] =df.progress_apply(converter, axis=1) 
confusion_matrix = pd.crosstab(df['predict'], df['context_label'], rownames=['Predicted'], colnames=['Actual'])
print(confusion_matrix)
result = (confusion_matrix[0][0]+confusion_matrix[1][1])/1700
print('accuracy:', result)

100%|██████████| 1700/1700 [00:00<00:00, 55596.15it/s]

Actual       0    1
Predicted          
0          616  137
1          234  713
accuracy: 0.7817647058823529


In [13]:
predict = df[df['predict'] == 0]
predict = predict[predict['bert_base_score'] < 0.5]
predict = predict[predict['context_label'] == 0]
predict

,img_local_path,caption1,caption2,context_label,article_url,maskrcnn_bboxes,caption1_modified,caption1_entities,caption2_modified,caption2_entities,bert_base_score,bert_large_score,nli_label,predict
105,test/105.jpg,Lamar preforming in 2013.,Kendrick Lamar in concert.,0,https://en.wikipedia.org/wiki/Kendrick_Lamar#/...,"[[689.6602172851562, 0.0, 1198.6527099609375, ...",GPE preforming in DATE.,"[[Lamar, GPE], [2013, DATE]]",PERSON in concert.,"[[Kendrick Lamar, PERSON]]",0.321324,0.428131,"{'label': 'ENTAILMENT', 'score': 0.73089224100...",0
182,test/182.jpg,The U. S. Supreme Court building stands quietl...,The U. S. Supreme Court building in Washington...,0,https://www.military.com/daily-news/2019/02/22...,"[[251.759033203125, 307.00665283203125, 283.73...",ORG building stands quietly before dawn in GPE...,"[[The U. S. Supreme Court, ORG], [Washington, ...","ORG building in GPE, D.C.","[[The U. S. Supreme Court, ORG], [Washington, ...",0.452076,0.436533,"{'label': 'ENTAILMENT', 'score': 0.86213296651...",0
226,test/226.jpg,Eight-year-old Noah saw this NHS signage on a ...,Boy watching truck,0,https://www.bbc.com/news/uk-scotland-52522179,"[[403.1940612792969, 245.27005004882812, 572.2...",DATE PERSON saw this NHS signage on a truck as...,"[[Eight-year-old, DATE], [Noah, PERSON], [dail...",Boy watching truck,[],0.344508,0.372085,"{'label': 'ENTAILMENT', 'score': 0.74134582281...",0
302,test/302.jpg,Alison Escobar's artistically taken shot of he...,Girl reflected in glass,0,https://www.bbc.com/news/uk-scotland-54058547,"[[243.41827392578125, 336.2706604003906, 488.1...",PERSON artistically taken shot of her DATE dau...,"[[Alison Escobar's, PERSON], [three-year-old, ...",Girl reflected in glass,[],0.319020,0.426092,"{'label': 'ENTAILMENT', 'score': 0.95468705892...",0
322,test/322.jpg,People pause to observe two minutes' silence a...,Waterloo station,0,https://www.bbc.com/news/in-pictures-54902358,"[[4.390007495880127, 133.0186004638672, 194.00...",People pause to observe TIME' silence at ORG i...,"[[two minutes, TIME], [Waterloo Station, ORG],...",Waterloo station,[],0.391593,0.463961,"{'label': 'ENTAILMENT', 'score': 0.80784636735...",0
328,test/328.jpg,"An image posted to Facebook on Jan. 21, 2017, ...",Kivalov was among those Ukrainians who attende...,0,https://www.nytimes.com/2019/01/10/us/politics...,"[[241.3650665283203, 106.92843627929688, 357.4...","An image posted to ORG on DATE, shows PERSON, ...","[[Facebook, ORG], [Jan. 21, 2017, DATE], [Serh...",ORG was among those NORP who attended the Libe...,"[[Kivalov, ORG], [Ukrainians, NORP]]",0.391195,0.552350,"{'label': 'ENTAILMENT', 'score': 0.66637045145...",0
363,test/363.jpg,Joe and Joshua appreciate a neighbour's rainbo...,rainbow children,0,https://www.bbc.com/news/uk-scotland-54616107,"[[357.75140380859375, 495.46649169921875, 600....",PERSON and PERSON appreciate a neighbour's rai...,"[[Joe, PERSON], [Joshua, PERSON]]",PERSON,"[[rainbow children, PERSON]]",0.228642,0.333510,"{'label': 'ENTAILMENT', 'score': 0.44314879179...",0
367,test/367.jpg,"For my next trick, a treat: Squirreling away s...",Squirrel,0,https://www.bbc.com/news/uk-scotland-54753318,"[[62.27503204345703, 339.3446350097656, 795.36...","For my next trick, a treat: Squirreling away s...","[[Halloween, DATE], [Lorraine Paton, PERSON]]",Squirrel,[],0.333601,0.295266,"{'label': 'ENTAILMENT', 'score': 0.48083561658...",0
376,test/376.jpg,"Ann Antonopoulou: ""Our Parson Russell Terrier ...",Dog under a cover,0,https://www.bbc.com/news/in-pictures-56015511,"[[54.76982498168945, 259.3638916015625, 957.48...","PERSON: ""Our PERSON new cave bed just arrived ...","[[Ann Antonopoulou, PERSON], [Parson Russell T...",Dog under a cover,[],0.279877,0.202844,"{'label': 'ENTAILMENT', 'score': 0.78115022182...",0
424,test/424.jpg,"Terry the Turtle flipping the bird, by Mark Fi...",A turtle appearing to be gesticulating at the ...,0,https://www.bbc.com/news/in-pictures-54118899,"[[119.8872299194336, 446.0697937011719, 8

In [19]:
tqdm.pandas()
converter = lambda x : 'CONTRADICTION' if (x.nli_label['label'] == 'CONTRADICTION' and x.nli_label_reverse['label'] != 'ENTAILMENT') or (x.nli_label_reverse['label'] == 'CONTRADICTION' and  x.nli_label['label'] != 'ENTAILMENT') else ('ENTAILMENT' if (x.nli_label['label'] == 'ENTAILMENT' and x.nli_label_reverse['label'] != 'CONTRADICTION') or (x.nli_label_reverse['label'] == 'ENTAILMENT' and x.nli_label['label'] != 'CONTRADICTION') else 'NEUTRAL')
df['nli'] = df.progress_apply(converter, axis=1) 

100%|██████████| 1700/1700 [00:00<00:00, 22919.55it/s]


In [30]:
tqdm.pandas()
converter = lambda x : 'CONTRADICTION' if (x.nli_label['label'] == 'CONTRADICTION' and x.nli_label_reverse['label'] == 'CONTRADICTION') else ('ENTAILMENT' if (x.nli_label['label'] == 'ENTAILMENT' or x.nli_label_reverse['label'] == 'ENTAILMENT') else 'NEUTRAL')
df['nli'] = df.progress_apply(converter, axis=1) 

100%|██████████| 1700/1700 [00:00<00:00, 25257.49it/s]


In [32]:
tqdm.pandas()
converter = lambda x : 'CONTRADICTION' if (x.nli_label['label'] == 'CONTRADICTION' or x.nli_label_reverse['label'] == 'CONTRADICTION') else ('ENTAILMENT' if (x.nli_label['label'] == 'ENTAILMENT' or x.nli_label_reverse['label'] == 'ENTAILMENT') else 'NEUTRAL')
df['nli'] = df.progress_apply(converter, axis=1) 

100%|██████████| 1700/1700 [00:00<00:00, 23851.60it/s]


In [33]:
converter = lambda x : 1 if x.nli == 'CONTRADICTION' else (0 if x.nli == 'ENTAILMENT' else (1 if x.bert_base_score < 0.5 else 0))
df['predict'] =df.progress_apply(converter, axis=1) 
confusion_matrix = pd.crosstab(df['predict'], df['context_label'], rownames=['Predicted'], colnames=['Actual'])
print(confusion_matrix)
result = (confusion_matrix[0][0]+confusion_matrix[1][1])/1700
print('accuracy:', result)

100%|██████████| 1700/1700 [00:00<00:00, 29711.02it/s]

Actual       0    1
Predicted          
0          593   60
1          257  790
accuracy: 0.8135294117647058


In [28]:
converter = lambda x : 0 if x.nli == 'ENTAILMENT' else 1
df['predict'] =df.progress_apply(converter, axis=1) 
confusion_matrix = pd.crosstab(df['predict'], df['context_label'], rownames=['Predicted'], colnames=['Actual'])
print(confusion_matrix)
result = (confusion_matrix[0][0]+confusion_matrix[1][1])/1700
print('accuracy:', result)

100%|██████████| 1700/1700 [00:00<00:00, 41437.26it/s]

Actual       0    1
Predicted          
0          324   27
1          526  823
accuracy: 0.6747058823529412


In [12]:
predict = df[df['predict'] == 0]
predict = predict[predict['context_label'] == 1]
predict

,img_local_path,caption1,caption2,context_label,article_url,maskrcnn_bboxes,caption1_modified,caption1_entities,caption2_modified,caption2_entities,bert_base_score,bert_large_score,nli_label,nli_label_reverse,nli,predict
49,test/49.jpg,The FBI raided the CDC in the middle of the ni...,A known hoax purveyor spread false claims that...,1,https://www.snopes.com/fact-check/fbi-raids-cd...,"[[641.717590332, 325.9614868164, 672.790588378...",The ORG raided the ORG in TIME to seize data o...,"[[FBI, ORG], [CDC, ORG], [the middle of the ni...",A known hoax purveyor spread false claims that...,"[[FBI, ORG], [CDC, ORG], [the middle of the ni...",0.692395,0.797202,"{'label': 'NEUTRAL', 'score': 0.7838650346}","{'label': 'ENTAILMENT', 'score': 0.6771703959}",ENTAILMENT,0
53,test/53.png,A photograph shows a genuine fossil of a megal...,While this image likely shows a genuine megalo...,1,https://www.snopes.com/fact-check/megalodon-to...,"[[137.7947387695, 10.0731515884, 455.678039550...",A photograph shows a genuine fossil of a megal...,[],While this image likely shows a genuine megalo...,"[[two, CARDINAL]]",0.712833,0.711089,"{'label': 'NEUTRAL', 'score': 0.8463303447}","{'label': 'ENTAILMENT', 'score': 0.7670401931}",ENTAILMENT,0
55,test/55.jpg,Trash being left behind by the migrant caravan.,This photograph was not recent and did not sho...,1,https://www.snopes.com/fact-check/refugee-cara...,"[[492.6638793945, 337.4176940918, 819.84039306...",Trash being left behind by the migrant caravan.,[],This photograph was not recent and did not sho...,"[[Honduras, GPE], [U.S, GPE], [Mexico, GPE], [...",0.339660,0.161456,"{'label': 'CONTRADICTION', 'score': 0.8995783925}","{'label': 'ENTAILMENT', 'score': 0.9085165858}",ENTAILMENT,0
76,test/76.jpg,A picture showing Goldie Hawn and Kurt Russel...,An image of actors Goldie Hawn and Kurt Russel...,1,https://www.snopes.com/fact-check/goldie-hawn-...,"[[157.2116088867, 8.691365242, 276.2471923828,...",A picture showing PERSON and PERSON both wear...,"[[Goldie Hawn, PERSON], [Kurt Russell, PERSON]...",An image of actors PERSON and PERSON wearing P...,"[[Goldie Hawn, PERSON], [Kurt Russell, PERSON]...",0.695945,0.723471,"{'label': 'NEUTRAL', 'score': 0.7818806171}","{'label': 'ENTAILMENT', 'score': 0.9588326216}",ENTAILMENT,0
200,test/200.jpg,Actors Goldie Hawn and Kurt Russell were spott...,A picture purportedly showing Goldie Hawn and ...,1,https://www.snopes.com/fact-check/goldie-hawn-...,"[[73.0175018311, 1.9323132038000002, 382.68859...",Actors Goldie Hawn and PERSON were spotted wea...,"[[Kurt Russell, PERSON], [Donald Trump, PERSON]]",A picture purportedly showing PERSON and PERSO...,"[[Goldie Hawn, PERSON], [Kurt Russell, PERSON]...",0.777599,0.736808,"{'label': 'NEUTRAL', 'score': 0.8834425807}","{'label': 'ENTAILMENT', 'score': 0.88628125190...",ENTAILMENT,0
204,test/204.jpg,California's governor is under investigation f...,The story California Governor Jerry Brown to a...,1,https://www.snopes.com/fact-check/jerry-brown-...,"[[304.3259277344, 311.4779052734, 357.18872070...",GPE's governor is under investigation for poss...,"[[California, GPE]]",The story GPE Governor PERSON to a corruption ...,"[[California, GPE], [Jerry Brown, PERSON]]",0.483953,0.461200,"{'label': 'CONTRADICTION', 'score': 0.50741523...","{'label': 'ENTAILMENT', 'score': 0.8702480197}",ENTAILMENT,0
224,test/224.png,A picture shows a massive tsunami headed for a...,The pictures of the huge tsunami hitting a cro...,1,https://www.snopes.com/fact-check/huge-tsunami...,"[[427.2711181641, 322.8975219727, 441.53894042...",A picture shows a massive tsunami headed for a...,[],The pictures of the huge tsunami hitting a cro...,"[[Honolulu, GPE]]",0.496911,0.476031,"{'label': 'NEUTRAL', 'score': 0.940289259}","{'label': 'ENTAILMENT', 'score': 0.96437042950...",ENTAILMENT,0
296,test/296.jpg,President Trump signed an executive order nami...,The post about Trump signeing an executive ord...,1,https://www.snopes.com/fact-check/trump-order-...

In [29]:
converter = lambda x : 1 if x.nli == 'CONTRADICTION' else 0
df['predict'] =df.progress_apply(converter, axis=1) 
confusion_matrix = pd.crosstab(df['predict'], df['context_label'], rownames=['Predicted'], colnames=['Actual'])
print(confusion_matrix)
result = (confusion_matrix[0][0]+confusion_matrix[1][1])/1700
print('accuracy:', result)

100%|██████████| 1700/1700 [00:00<00:00, 46774.27it/s]

Actual       0    1
Predicted          
0          833  588
1           17  262
accuracy: 0.6441176470588236


In [14]:
predict = df[df['predict'] == 1]
predict = predict[predict['context_label'] == 0]
predict

,img_local_path,caption1,caption2,context_label,article_url,maskrcnn_bboxes,caption1_modified,caption1_entities,caption2_modified,caption2_entities,bert_base_score,bert_large_score,nli_label,nli_label_reverse,nli,predict
19,test/19.jpg,"Asylum seekers lining up last week in Tijuana,...",Asylum seekers lined up in January outside the...,0,https://www.nytimes.com/issue/todayspaper/2019...,"[[358.8293151855, 146.7284393311, 452.78509521...","Asylum seekers lining up DATE in GPE, GPE.","[[last week, DATE], [Tijuana, GPE], [Mexico, G...",Asylum seekers lined up in DATE outside the OR...,"[[January, DATE], [El Chaparral, ORG], [the Un...",0.792984,0.762001,"{'label': 'NEUTRAL', 'score': 0.8978046179}","{'label': 'CONTRADICTION', 'score': 0.8740615249}",CONTRADICTION,1
20,test/20.jpg,"Martin Fayulu, the opposition leader and runne...",Opposition presidential candidate in the Democ...,0,https://www.nytimes.com/2019/01/19/world/afric...,"[[141.8394775391, 26.6930541992, 307.372161865...","PERSON, the opposition leader and runner-up in...","[[Martin Fayulu, PERSON]]",Opposition presidential candidate in GPE Marti...,"[[the Democratic Republic of Congo, GPE], [Kin...",0.463729,0.574304,"{'label': 'NEUTRAL', 'score': 0.9806368947}","{'label': 'CONTRADICTION', 'score': 0.6219409108}",CONTRADICTION,1
65,test/65.jpg,Policemen stand guard near the state secretari...,"Defying Court Order, Police Bar 10 Women from ...",0,https://www.voanews.com/south-central-asia/pro...,"[[93.2202072144, 136.4102630615, 232.829162597...",Policemen stand guard near the state secretari...,"[[two, CARDINAL], [Sabarimala, GPE], [Hindu, N...","Defying Court Order, Police Bar 10 Women from ...","[[Indian, NORP]]",0.561645,0.527768,"{'label': 'CONTRADICTION', 'score': 0.90436863...","{'label': 'NEUTRAL', 'score': 0.5590627789}",CONTRADICTION,1
67,test/67.jpg,A member of a feminist collective paints the h...,A demonstrator paints the helmet visor of a ri...,0,https://www.bbc.com/news/in-pictures-55150152,"[[8.3900680542, 63.5647697449, 528.44140625, 6...",A member of a feminist collective paints the h...,"[[Mexico City, GPE], [Mexico, GPE], [November ...",A demonstrator paints the helmet visor of a ri...,"[[Mexico City, GPE], [Mexico, GPE], [11 Novemb...",0.693692,0.772994,"{'label': 'CONTRADICTION', 'score': 0.5625689626}","{'label': 'NEUTRAL', 'score': 0.8131160736}",CONTRADICTION,1
97,test/97.jpg,An old man leans up against his truck in Granby.,East Troublesome fire grows as residents evacu...,0,https://www.denverpost.com/2020/10/22/photos-e...,"[[325.714630127, 173.3758239746, 395.441223144...",An old man leans up against his truck in GPE.,"[[Granby, GPE]]",East Troublesome fire grows as residents evacu...,"[[Granby, GPE]]",0.147661,0.265497,"{'label': 'CONTRADICTION', 'score': 0.77687680...","{'label': 'CONTRADICTION', 'score': 0.65027189...",CONTRADICTION,1
99,test/99.jpg,A normally busy street near Potsdamer Platz on...,"Corona bans have turned the German capital, Be...",0,https://www.nytimes.com/2020/04/16/briefing/co...,"[[313.431060791, 305.7563171387, 359.409820556...",A normally busy street near FAC on DATE in GPE.,"[[Potsdamer Platz, FAC], [Tuesday, DATE], [Ber...","Corona bans have turned the NORP capital, GPE ...","[[German, NORP], [Berlin, GPE]]",0.287640,0.218119,"{'label': 'CONTRADICTION', 'score': 0.5363262296}","{'label': 'NEUTRAL', 'score': 0.6792238355}",CONTRADICTION,1
101,test/101.jpg,"In December, the police kept watch over a prot...",A drawing depicting Hungarian President Viktor...,0,https://www.google.com/url?sa=i&url=https%3A%2...,"[[112.9863739014, 16.8551254272, 466.530120849...","In DATE, the police kept watch over a protest ...","[[December, DATE], [Viktor Orban, PERSON]]",A drawing depicting NORP President PERSON is p...,"[[Hungarian, NORP], [Viktor Orban, PERSON]]",0.407983,0.451548,"{'label': 'NEUTRAL', 'score': 0.8208233118}","{'label': 'CONTRADICTION', 'score': 0.5079464316}",CONTRADICTION,1
133,test/133.jpg,"Mr. Trump and Mr. Puti